# Le choix d'un SGBD (Relationnel vs NoSQL)
## Objectif

Ce notebook simule le parcours d'un développeur qui doit créer un *blog*.

**Note** : Pour que ce notebook soit simple d'utilisation et pour qu'il soit exécutable par tous sans installation de serveur, nous utiliserons `sqlite3` pour simuler le SQL et des objets Python natifs pour simuler le comportement "Document" de MongoDB. Cela permet de diffuser plus simplement ce notebook à d'autre développeur voulant se former aux différentes approches.

## Partie 1 : L'approche Relationnelle (PostgreSQL/SQL)

Nous commençons donc par une approche relationnelle, car c'est le système que nous connaissons le plus et cette solution parait être la plus simple.

### Étape 1.1 : Création du schéma (Articles, Auteurs, Commentaires)

In [2]:
import sqlite3
import pandas as pd

# Création d'une base de données
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# 1. Création des tables
cursor.executescript('''
    CREATE TABLE users (
        id INTEGER PRIMARY KEY,
        username TEXT NOT NULL,
        email TEXT NOT NULL
    );

    CREATE TABLE articles (
        id INTEGER PRIMARY KEY,
        title TEXT NOT NULL,
        content TEXT,
        author_id INTEGER,
        FOREIGN KEY(author_id) REFERENCES users(id)
    );

    CREATE TABLE comments (
        id INTEGER PRIMARY KEY,
        article_id INTEGER,
        user_id INTEGER,
        content TEXT,
        parent_comment_id INTEGER NULL,
        FOREIGN KEY(article_id) REFERENCES articles(id),
        FOREIGN KEY(user_id) REFERENCES users(id),
        FOREIGN KEY(parent_comment_id) REFERENCES comments(id)
    );
''')

# 2. Insertion de données
cursor.executescript('''
    INSERT INTO users (id, username, email) VALUES (1, 'Alice', 'alice@blog.com');
    INSERT INTO users (id, username, email) VALUES (2, 'Bob', 'bob@blog.com');

    INSERT INTO articles (id, title, content, author_id) VALUES (1, 'Pourquoi choisir le bon SGBD ?', 'Relationnel ou non-relationnel ?', 1);

    INSERT INTO comments (id, article_id, user_id, content, parent_comment_id) VALUES (1, 1, 1, 'Super article !', NULL);

    INSERT INTO comments (id, article_id, user_id, content, parent_comment_id) VALUES (2, 1, 2, 'Pas d''accord avec toi.', 1);

    INSERT INTO comments (id, article_id, user_id, content, parent_comment_id) VALUES (3, 1, 1, 'Ah bon ? Pourquoi ?', 2);
''')

print("[-] Base SQL initialisée avec succès.")

[-] Base SQL initialisée avec succès.


### Étape 1.2 : La Récursivité

Le client demande d'afficher les commentaires sous forme d'arbre. En SQL, récupérer une hiérarchie nécessite d'utiliser des CTE. C'est très efficace, mais verbeux et complexe à maintenir pour un débutant.

La gestion des commentaires imbriqués sous forme d'arbre (avec récursivité) permet de modéliser des hiérarchies complexes.

In [4]:
# Sélection des commentaires

query_recursive = '''
WITH RECURSIVE comment_tree AS (
    SELECT id, user_id, content, parent_comment_id, 0 as level, content as path
    FROM comments
    WHERE parent_comment_id IS NULL AND article_id = 1

    UNION ALL

    SELECT c.id, c.user_id, c.content, c.parent_comment_id, ct.level + 1, ct.path || ' > ' || c.content
    FROM comments c
    INNER JOIN comment_tree ct ON c.parent_comment_id = ct.id
)
SELECT level, path FROM comment_tree;
'''

df = pd.read_sql_query(query_recursive, conn)
print("Résultat de la requête récursive SQL :")
print(df)

Résultat de la requête récursive SQL :
   level                                               path
0      0                                    Super article !
1      1           Super article ! > Pas d'accord avec toi.
2      2  Super article ! > Pas d'accord avec toi. > Ah ...


On constate que cette première requête est déjà très complexe pour seulement afficher les commentaires. Si il y a 10 niveaux d'imbrications le résultat va être lourd et complexe à exploiter. Si le schéma évolue, cette requête risque de ne plus fonctionner...

On peut essayer d'utiliser du **NoSQL**, on a un besoin de *flexibilité* pour ce besoin.

## Partie 2 : La migration vers le NoSQL

On décide de passer à MongoDB. L'argument principal : la flexibilité du schéma et la gestion native des données imbriquées. Plus besoin de jointures, on stocke tout l'article et ses commentaires dans un seul document JSON.

### Étape 2.1 : La structure "Document"

Ici, nous simulons une collection MongoDB avec des dictionnaires Python (la structure est exactement la même). En Mongo, il est très facile de représenter l'article et ses commentaires imbriqués.

In [5]:
import json

# Simulation de la base MongoDB (Collection 'posts')
mongo_db_posts = []

# Création d'un document Article avec les commentaires (Dénormalisation)
new_post = {
    "_id": "post_1",
    "title": "Pourquoi choisir le bon SGBD ?",
    "content": "Relationnel ou non-relationnel ?",
    "author": {
        "user_id": 1,
        "username": "Alice"
    },
    "comments": [
        {
            "comment_id": 1,
            "user": { "id": 1, "username": "Alice" },
            "content": "Super article !",
            "replies": [
                {
                    "comment_id": 2,
                    "user": { "id": 2, "username": "Bob" },
                    "content": "Pas d'accord avec toi.",
                    "replies": [
                        {
                            "comment_id": 3,
                            "user": { "id": 1, "username": "Alice" },
                            "content": "Ah bon ? Pourquoi ?",
                            "replies": []
                        }
                    ]
                }
            ]
        }
    ]
}

mongo_db_posts.append(new_post)

print("[-] Document inséré dans 'MongoDB'.")
print("Lecture de l'article et de ses commentaires :")
# En Mongo, un simple find_one() suffit !
print(json.dumps(mongo_db_posts[0]['comments'], indent=2))

[-] Document inséré dans 'MongoDB'.
Lecture de l'article et de ses commentaires :
[
  {
    "comment_id": 1,
    "user": {
      "id": 1,
      "username": "Alice"
    },
    "content": "Super article !",
    "replies": [
      {
        "comment_id": 2,
        "user": {
          "id": 2,
          "username": "Bob"
        },
        "content": "Pas d'accord avec toi.",
        "replies": [
          {
            "comment_id": 3,
            "user": {
              "id": 1,
              "username": "Alice"
            },
            "content": "Ah bon ? Pourquoi ?",
            "replies": []
          }
        ]
      }
    ]
  }
]


Super, nous avons trouvé un moyen simple et efficace d'obtenir une hiérarchie de commentaire !
En plus, lorsque nous récupérons un commentaire nous avons directement accès à ses fils, son id, son user, etc.

## Partie 3 : Est-ce la solution parfaite ?

Le projet avance et une nouvelle demande métier arrive : *L'utilisateur "Alice" veut changer son pseudo pour "Alice_du_35"*.

### Étape 3.1 : Le problème de la dénormalisation

In [6]:
# Scénario : Alice change de nom
nouveau_pseudo = "Alice_du_35"

print(f"--- Mise à jour du pseudo : 'Alice' -> '{nouveau_pseudo}' ---\n")

# 1. En SQL (Simulation)
cursor.execute("UPDATE users SET username = ? WHERE id = 1", (nouveau_pseudo,))
# Vérification SQL
verif_sql = pd.read_sql_query("SELECT username FROM users WHERE id=1", conn)
print(f"SQL : Mise à jour effectuée en 1 opération. Nouvelle valeur : {verif_sql.iloc[0]['username']}")


# 2. En NoSQL (Simulation de la complexité)
# Nous devons parcourir tous les articles, et dans chaque article, tous les commentaires (et sous-commentaires...)
def update_nested_comments(comments, user_id, new_name):
    count = 0
    for comment in comments:
        if comment['user']['id'] == user_id:
            comment['user']['username'] = new_name
            count += 1
        count += update_nested_comments(comment['replies'], user_id, new_name)
    return count

total_updates_mongo = 0
for post in mongo_db_posts:
    if post['author']['user_id'] == 1:
        post['author']['username'] = nouveau_pseudo
        total_updates_mongo += 1

    total_updates_mongo += update_nested_comments(post['comments'], 1, nouveau_pseudo)

print(f"NoSQL : Mise à jour effectuée. Nombre de documents/objets modifiés : {total_updates_mongo}")

--- Mise à jour du pseudo : 'Alice' -> 'Alice_du_35' ---

SQL : Mise à jour effectuée en 1 opération. Nouvelle valeur : Alice_du_35
NoSQL : Mise à jour effectuée. Nombre de documents/objets modifiés : 3


Dans notre base SQL, nous changeons seulement le nom dans la table users et c'est fini (une seule ligne modifiée). Mais en NoSQL (MongoDB), nous avons copié (dénormalisé) le pseudo d'Alice dans chaque commentaire et dans chaque article qu'elle a écrit pour gagner en performance de lecture.

La dénormalisation peut entraîner des redondances ou des incohérences si elle n'est pas maîtrisée. En plus d'être lourd, il y a toujours un risque d'oublier des collections ou des éléments.

Pour un simple changement de nom, on a dû écrire un script de plusieurs lignes qui lit toute la base. Si le script est interrompu, certains commentaires seront écrit par "Alice" et d'autre par "Alice_du_35".

## Conclusion

Ce notebook illustre les enseignements de ce projet :

1. PostgreSQL (SQL) : Idéal quand l'intégrité des données est cruciale. Les modifications sont sûres (ACID), mais la lecture de données complexes (arbres) demande des requêtes techniques difficiles.

2. MongoDB (NoSQL) : Idéal pour démarrer vite (POC) et lire des données imbriquées rapidement. Mais la maintenance des données (mises à jour) devient lourde si les données trop dénormalisé.

La solution "Hybride" ?

Comme mentionné dans le rendu, PostgreSQL avec JSONB est souvent le bon compromis. On garde la table Users propre (Relationnel) et on stocke le contenu plus complexes comme les commentaires en JSONB pour la flexibilité, sans dupliquer le nom de l'utilisateur.

De plus, un travail de veille est necessaire car il existe probablement un SGBD adapté pour gérer des problématiques similaires (blogs ou conversation).

Voici la phrase que nous pouvons retenir de ce projet :

    "Le SGBD optimal n'existe pas : il est spécifique à chaque projet."

Ce Notebook a pour but de sensibiliser les développeurs à prendre le temps d'étudier les différentes possibilités avant de se lancer en suivant son intuition. Nous avons vu ici que de se lancer trop vite en utilisant une technologie en particulier peut nous faire perdre énormément de temps.
Ce sujet nous semble très important d'autant plus que dans moins d'un an, nous rentrerons dans le monde professionnel, nous devons être capables de fournir la meilleure solution à un projet et être capable de préciser pourquoi une solution est mieux que l'autre, tout en anticipant les besoins futurs d'un projet afin de le rendre pérenne dans le temps.